# LAPD Crime Analysis

In [18]:
# import packages
import os
import pandas as pd
import warnings
from google.colab import drive
drive.mount('/content/drive')

pd.set_option('display.max_columns', None) # display all columns when df is called
warnings.filterwarnings("ignore") # ignore all warnings

Mounted at /content/drive


Data source of truth: https://catalog.data.gov/dataset/crime-data-from-2020-to-present

In [25]:
# load data from csv
data_dir = '/content/drive/MyDrive/DAT 490 - Forest Flux/data/Crime_Data_from_2020_to_Present.csv'
df = pd.read_csv(data_dir)

In [26]:
df.head(3)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,211507896,04/11/2021 12:00:00 AM,11/07/2020 12:00:00 AM,845,15,N Hollywood,1502,2,354,THEFT OF IDENTITY,0377,31,M,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,7800 BEEMAN AV,NaN,34.2124,-118.4092
1,201516622,10/21/2020 12:00:00 AM,10/18/2020 12:00:00 AM,1845,15,N Hollywood,1521,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0416 0334 2004 1822 1414 0305 0319 0400,32,M,H,102.0,SIDEWALK,200.0,KNIFE WITH BLADE 6INCHES OR LESS,IC,Invest Cont,230.0,NaN,NaN,NaN,ATOLL AV,N GAULT,34.1993,-118.4203
2,240913563,12/10/2024 12:00:00 AM,10/30/2020 12:00:00 AM,1240,9,Van Nuys,933,2,354,THEFT OF IDENTITY,0377,30,M,W,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,14600 SYLVAN ST,NaN,34.1847,-118.4509


## Descriptive analysis and data preparation

In [27]:
# Numberof records in the datafreame:
df.shape

(1004991, 28)

**Make all column names lowercase**

In [ ]:
# Make all columns lowercase
df.columns = df.columns.str.replace(" ", "_").str.lower()

In [ ]:
df.head(1)

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
0,211507896,04/11/2021 12:00:00 AM,11/07/2020 12:00:00 AM,845,15,N Hollywood,1502,2,354,THEFT OF IDENTITY,0377,31,M,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,7800 BEEMAN AV,NaN,34.2124,-118.4092


In [ ]:
# Data collection time range
print("Oldest reported record in the dataframe:", pd.to_datetime(df["date_rptd"], errors="coerce").dt.date.sort_values(ascending=True).head(1))
print("Most recent reported record in the dataframe:", pd.to_datetime(df["date_rptd"], errors="coerce").dt.date.sort_values(ascending=False).head(1))

Oldest reported record in the dataframe: 149605    2020-01-01
Name: date_rptd, dtype: object
Most recent reported record in the dataframe: 1004911    2025-06-04
Name: date_rptd, dtype: object


**Make all object/string records lowercase**

In [ ]:
# select all object type columns
df.select_dtypes(include="object").head(1)

,date_rptd,date_occ,area_name,crm_cd_desc,mocodes,vict_sex,vict_descent,premis_desc,weapon_desc,status,status_desc,location,cross_street
0,04/11/2021 12:00:00 AM,11/07/2020 12:00:00 AM,N Hollywood,THEFT OF IDENTITY,0377,M,H,SINGLE FAMILY DWELLING,NaN,IC,Invest Cont,7800 BEEMAN AV,NaN


In [ ]:
string_column_names = df.select_dtypes(include="object").columns # list of string columns
# make all string_column_names lowercase and re-assign them to the original dataframe
df[string_column_names] = df[string_column_names].astype(str).applymap(str.lower)
df.select_dtypes(include="object").head(1)

,date_rptd,date_occ,area_name,crm_cd_desc,mocodes,vict_sex,vict_descent,premis_desc,weapon_desc,status,status_desc,location,cross_street
0,04/11/2021 12:00:00 am,11/07/2020 12:00:00 am,n hollywood,theft of identity,0377,m,h,single family dwelling,nan,ic,invest cont,7800 beeman av,nan


**Numer of observations per year**

In [ ]:
# Extract date attributes from date_rptd and date_Occ columns. (Adding 6 new columns)
date_cols = ["date_rptd", "date_occ"]
new_date_cols = []

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")
    new_date_cols.append(col)
    df[f"{col}_year"] = df[col].dt.year
    new_date_cols.append(f"{col}_year")
    df[f"{col}_month"] = df[col].dt.month
    new_date_cols.append(f"{col}_month")
    df[f"{col}_day"] = df[col].dt.day
    new_date_cols.append(f"{col}_day")

print("Added 6 new columns")
df[new_date_cols].head(1)

Added 6 new columns


,date_rptd,date_rptd_year,date_rptd_month,date_rptd_day,date_occ,date_occ_year,date_occ_month,date_occ_day
0,2021-04-11,2021,4,11,2020-11-07,2020,11,7


In [ ]:
# Numer of observations per year
pd.DataFrame(df.groupby(by=["date_rptd_year"]).count()['dr_no']).rename(columns={"dr_no":"record count"})

,record count
date_rptd_year,
2020,192708
2021,208285
2022,235067
2023,234636
2024,134104
2025,191


**The average, median, min, and max of all numeric variables**

In [ ]:
df.describe()

,dr_no,date_rptd,date_occ,time_occ,area,rpt_dist_no,part_1-2,crm_cd,vict_age,premis_cd,weapon_used_cd,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,lat,lon,date_rptd_year,date_rptd_month,date_rptd_day,date_occ_year,date_occ_month,date_occ_day
count,1.004991e+06,1004991,1004991,1.004991e+06,1.004991e+06,1.004991e+06,1.004991e+06,1.004991e+06,1.004991e+06,1.004975e+06,327247.000000,1.004980e+06,69160.000000,2314.000000,64.00000,1.004991e+06,1.004991e+06,1.004991e+06,1.004991e+06,1.004991e+06,1.004991e+06,1.004991e+06,1.004991e+06
mean,2.202215e+08,2022-05-25 22:02:46.977017856,2022-05-13 17:50:37.556754944,1.339900e+03,1.069174e+01,1.115633e+03,1.400348e+00,5.001568e+02,2.891706e+01,3.056201e+02,363.955300,4.999174e+02,958.101258,984.015990,991.21875,3.399821e+01,-1.180909e+02,2.021910e+03,6.386917e+00,1.558349e+01,2.021879e+03,6.370997e+00,1.534594e+01
min,8.170000e+02,2020-01-01 00:00:00,2020-01-01 00:00:00,1.000000e+00,1.000000e+00,1.010000e+02,1.000000e+00,1.100000e+02,-4.000000e+00,1.010000e+02,101.000000,1.100000e+02,210.000000,310.000000,821.00000,0.000000e+00,-1.186676e+02,2.020000e+03,1.000000e+00,1.000000e+00,2.020000e+03,1.000000e+00,1.000000e+00
25%,2.106169e+08,2021-04-21 00:00:00,2021-04-06 00:00:00,9.000000e+02,5.000000e+00,5.870000e+02,1.000000e+00,3.310000e+02,0.000000e+00,1.010000e+02,311.000000,3.310000e+02,998.000000,998.000000,998.00000,3.401470e+01,-1.184305e+02,2.021000e+03,3.000000e+00,8.000000e+00,2.021000e+03,3.000000e+00,7.000000e+00
50%,2.209159e+08,2022-06-08 00:00:00,2022-05-26 00:00:00,1.420000e+03,1.100000e+01,1.139000e+03,1.000000e+00,4.420000e+02,3.000000e+01,2.030000e+02,400.000000,4.420000e+02,998.000000,998.000000,998.00000,3.405890e+01,-1.183225e+02,2.022000e+03,6.000000e+00,1.600000e+01,2.022000e+03,6.000000e+00,1.500000e+01
75%,2.311103e+08,2023-07-04 00:00:00,2023-06-22 00:00:00,1.900000e+03,1.600000e+01,1.613000e+03,2.000000e+00,6.260000e+02,4.400000e+01,5.010000e+02,400.000000,6.260000e+02,998.000000,998.000000,998.00000,3.416490e+01,-1.182739e+02,2.023000e+03,9.000000e+00,2.300000e+01,2.023000e+03,9.000000e+00,2.300000e+01
max,2.521041e+08,2025-06-04 00:00:00,2025-05-29 00:00:00,2.359000e+03,2.100000e+01,2.199000e+03,2.000000e+00,9.560000e+02,1.200000e+02,9.760000e+02,516.000000,9.560000e+02,999.000000,999.000000,999.00000,3.433430e+01,0.000000e+00,2.025000e+03,1.200000e+01,3.100000e+01,2.025000e+03,1.200000e+01,3.100000e+01
std,1.319718e+07,NaN,NaN,6.510613e+02,6.110255e+00,6.111605e+02,4.899691e-01,2.052731e+02,2.199272e+01,2.193021e+02,123.734528,2.050736e+02,110.354348,52.350982,27.06985,1.610713e+00,5.582386e+00,1.317239e+00,3.460478e+00,8.831701e+00,1.315051e+00,3.463282e+00,8.950559e+00


**Missing values, where and how many**

In [ ]:
missing_df = df.isna().sum().reset_index()
missing_df.columns = ["Column", "Missing_Count"]
missing_df[missing_df["Missing_Count"] > 0]

,Column,Missing_Count
14,premis_cd,16
16,weapon_used_cd,677744
20,crm_cd_1,11
21,crm_cd_2,935831
22,crm_cd_3,1002677
23,crm_cd_4,1004927


**Duplicated rows, where and how many**

In [ ]:
print("Duplicated rows")
# Boolean mask of duplicate rows
df[df.duplicated(keep=False)]


Duplicated rows


,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon,date_rptd_year,date_rptd_month,date_rptd_day,date_occ_year,date_occ_month,date_occ_day


**Save new df to csv**

In [ ]:
# df.to_csv("Crime_Data_prepared.csv", index=False)



**Observations:**

- Number of records in the dataframe: `1004991` rows and `28` columns
- Data collection time range: From `2020-01-01` to `2025-06-04`
- Numer of observations per year: `2020 -> 192708`, `2021	-> 208285`, `2022 -> 235067`, `2023	-> 234636`, `2024 -> 134104`, `2025 -> 191`
- Missing values, Where and how many: `crm_cd_1`, `crm_cd_2`, `crm_cd_3`, `crm_cd_4`, `weapon_used_cd`, `premis_cd`
- Duplicated rows, where and how many: `None`
  
**Actions**

- Make all column names lowercase.
- Make all object/string records lowercase
- create columns: `date_rptd, date_rptd_year, date_rptd_month, date_rptd_day, date_occ, date_occ_year, date_occ_month, date_occ_day`
